In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install datasets transformers

In [24]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
pip install --upgrade transformers datasets

In [4]:
from datasets import load_dataset
from transformers import BartTokenizer, BartForConditionalGeneration
from transformers import TrainingArguments, Trainer, pipeline

2025-05-09 12:40:35.308809: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746794435.333173     187 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746794435.340356     187 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
dataset = load_dataset("cnn_dailymail", "3.0.0")

In [6]:
df = pd.DataFrame(dataset['train'][:5])  
print(df.head())

                                             article  \
0  LONDON, England (Reuters) -- Harry Potter star...   
1  Editor's note: In our Behind the Scenes series...   
2  MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...   
3  WASHINGTON (CNN) -- Doctors removed five small...   
4  (CNN)  -- The National Football League has ind...   

                                          highlights  \
0  Harry Potter star Daniel Radcliffe gets £20M f...   
1  Mentally ill inmates in Miami are housed on th...   
2  NEW: "I thought I was going to die," driver sa...   
3  Five small polyps found during procedure; "non...   
4  NEW: NFL chief, Atlanta Falcons owner critical...   

                                         id  
0  42c027e4ff9730fbb3de84c1af0d2c506e41c3e4  
1  ee8871b15c50d0db17b0179a6d2beab35065f1e9  
2  06352019a19ae31e527f37f7571c6dd7f0c5da37  
3  24521a2abb2e1f5e34e6824e0f9e56904a2b0e88  
4  7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a  


In [7]:
sample = dataset['train'][0]
print("Article:", sample['article'])
print("Summary:", sample['highlights'])

Article: LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Detai

In [31]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

def tokenize_function(example):
    model_inputs = tokenizer(
        example["article"],
        max_length=1024,
        truncation=True,
        padding="max_length"
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example["highlights"],
            max_length=128,
            truncation=True,
            padding="max_length"
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

small_dataset = dataset["train"].select(range(10000))
tokenized_dataset = small_dataset.map(tokenize_function, batched=True, num_proc=4)
train_dataset = tokenized_dataset.select(range(10000))


Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your 

In [32]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

training_args = TrainingArguments(
    output_dir="./results",
    # evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    weight_decay=0.01,
    save_total_limit=1,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
500,0.773900
1000,0.588800
1500,0.570600
2000,0.553100
2500,0.548200


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:71: UserWarnin

TrainOutput(global_step=2500, training_loss=0.606921826171875, metrics={'train_runtime': 3792.1904, 'train_samples_per_second': 2.637, 'train_steps_per_second': 0.659, 'total_flos': 2.167104602112e+16, 'train_loss': 0.606921826171875, 'epoch': 1.0})

In [37]:
article = """
Renewable energy sources are transforming how the world generates electricity, offering cleaner and more sustainable alternatives to fossil fuels. Solar panels and wind turbines are becoming increasingly efficient and cost-effective, making them viable options for large-scale energy production. Innovations like improved battery storage and smart grids are enhancing the reliability of renewable energy, ensuring a steady power supply even when the sun isn't shining or the wind isn't blowing.
"""

In [38]:
import torch

inputs = tokenizer([article], max_length=512, truncation=True, return_tensors='pt')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
inputs = {k: v.to(device) for k, v in inputs.items()}

# Generate summary
summary_ids = model.generate(
    inputs['input_ids'], 
    num_beams=4, 
    max_length=50,  
    min_length=20,  
    length_penalty=2.0,  
    early_stopping=True
)

# Decode summary
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("Summary:", summary)

Summary: Renewable energy sources are transforming how the world generates electricity .
Solar panels and wind turbines are becoming increasingly efficient and cost-effective .
Smart grids are enhancing the reliability of renewable energy .


In [ ]:
!pip install evaluate
!pip install rouge-score

In [47]:
from evaluate import load
rouge = load("rouge")

reference_summary = ["Renewable energy sources are transforming electricity generation by offering sustainable alternatives and improving efficiency with innovations like battery storage."]
predictions = [summary]

rouge_score = rouge.compute(predictions=predictions, references=reference_summary)
print(rouge_score)

{'rouge1': 0.28, 'rouge2': 0.16666666666666669, 'rougeL': 0.28, 'rougeLsum': 0.28}


In [48]:
!pip install pyngrok

In [50]:
!ngrok config add-authtoken 2wriHqhlHtxI3EJ0wBL5wbU0pVf_4YxC2Bsn3ymVcCkV2R12q

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [56]:
from flask import Flask, request, jsonify, render_template_string
import torch
from transformers import BartForConditionalGeneration, BartTokenizer
from pyngrok import ngrok

app = Flask(__name__)

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

@app.route('/')
def index():
    return render_template_string("""
        <html>
            <body>
                <h1>Text Summarization API</h1>
                <form action="/predict" method="POST">
                    <textarea name="article" rows="5" cols="50" placeholder="Enter your article here..."></textarea><br>
                    <input type="submit" value="Generate Summary">
                </form>
            </body>
        </html>
    """)

# Route to predict the summary
@app.route('/predict', methods=['POST'])
def predict():
    article = request.form['article']  # Get article from the form

    # Process the article
    inputs = tokenizer([article], max_length=512, truncation=True, return_tensors='pt')
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        summary_ids = model.generate(
            inputs['input_ids'],
            num_beams=8,
            no_repeat_ngram_size=3, 
            max_length=60,
            min_length=20,
            length_penalty=2.0,
            early_stopping=True
        )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # Return the summary in the page
    return render_template_string("""
        <html>
            <body>
                <h1>Text Summarization API</h1>
                <h2>Original Article:</h2>
                <p>{{ article }}</p>
                <h2>Summary:</h2>
                <p>{{ summary }}</p>
                <br>
                <a href="/">Go back</a>
            </body>
        </html>
    """, article=article, summary=summary)

public_url = ngrok.connect(5000)
print('Flask app is running at:', public_url)

app.run(port=5000)


Flask app is running at: NgrokTunnel: "https://885c-34-60-133-205.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off
